In [16]:
import collections
import trees
import pickle

with open('sample.vars','rb') as f:
    # These sample variables are the result of treating the first 3 sentences as our entire training data
    sample_vars = pickle.load(f)

def read_trees(fname):
    """Read in all trees from a given file
    input: filename
    outputs: rule lookup dictionary, count of each rule seen"""
    # Replace this with your code, for now I put a placeholder in
    rules_lookup = sample_vars[0]
    rule_counts = sample_vars[1]
    return rules_lookup, rule_counts

fname = 'train.trees.pre.unk'
rules_lookup, rule_counts = read_trees(fname)
''' Format of rules_lookup provided is:
    key = RHS of rule
    value = set of valid LHS of rules
    For example:
    rules_lookup['the'] = {'DT'} corresponds to rule DT -> the
    rules_lookup['stop'] = {'VBP','NN'} corresponds to rules stop -> VBP and stop -> NN'''


''' Format of rule_counts provided is:
    key = rule as tuple of (LHS, RHS)
    value = how many times that rule was seen in the data
    For example:
    rule_counts[('PP', ('IN', 'NP_NNP'))] = 4
    rule_counts[('DT', 'this')] = 1'''
print()

In [20]:
rules_lookup['DT', 'NN']

{'NP'}

In [22]:
rule_counts['NP',('DT','NN')]

3

In [4]:
rule_counts['VBP','stop']

1

In [5]:
def get_probs(rules_lookup, rule_counts):
    """Given the rules_lookup dictionary and the total count of rules
    return a dictionary with keys as rules and values as probabilities"""
    grammar = sample_vars[2]
    return grammar

grammar = get_probs(rules_lookup, rule_counts)
''' Format of grammar provided is:
    key = rule as tuple of (LHS, RHS)
    value = conditional probability of RHS given LHS
    For example:
    grammar[('PP', ('IN', 'NP_NNP'))] = 0.66666666
    grammar[('DT', 'this')] = 0.25'''
print()

In [6]:
grammar['NN','stop']

0.3333333333333333

In [7]:
grammar['VBP','stop']

0.5

In [8]:
def CKY(sent, grammar):
    """Given a space separated sentence and your grammar,
    run CKY to fill the chart with the highest probability partial parses.
    Return the filled in chart from CKY"""
    chart = sample_vars[3]
    return chart

sent = sample_vars[4]
chart = CKY(sent, grammar)
''' Format of chart provided is:
    chart[row][column] = dictionary with:
        key = parse for that span (LHS of rule applied)
        value = [weight, RHS of rule applied, index of first word  of span (i), split index (x), diagonal # (diagonals)]
    For example:
    chart[0][0]['VBZ'] = [1.0, 'Does', 0, None, None]
    weight is 1.0, RHS of rule is Does, comes from word 0, there is no split, there is no diagonal #
    
    chart[0][-1]['TOP'] = [0.000992063492063492, ('SQ', 'PUNC'), 0, 4, 5]
    weight is 0.00099, RHS of rule is ('SQ', 'PUNC'), starts at word 0, splits after word index 4, 5th diagonal processed
    '''
print()

In [9]:
sent

'Does this flight serve dinner ?'

In [24]:
chart[0][-1]

{'TOP': [0.000992063492063492, ('SQ', 'PUNC'), 0, 4, 5]}

In [25]:
weight, rule, i, x, diagonals = chart[0][-1]['TOP']
print('Weight of this parse ',weight)
print('Rule applied to get here ', rule)
print('Left child entry ', chart[i][i+x])
print('Right child entry ', chart[i+x+1][i+diagonals])

Weight of this parse  0.000992063492063492
Rule applied to get here  ('SQ', 'PUNC')
Left child entry  {'SQ': [0.008928571428571428, ('VBZ', 'SQ*'), 0, 0, 4]}
Right child entry  {'PUNC': [0.3333333333333333, '?', 5, None, None]}


In [12]:
def parse(chart, s='TOP', row=0, col=-1):
    """Given the following:
    chart - filled in chart from CKY
    s - used for recursion, starts with 'TOP'
    row - the index of the row that i appears in
    col - the index of the column that s appears in
    This will be recursive, with s row and col changing"""
    parse_string = sample_vars[5]
    return parse_string

parse_string = parse(chart)
''' Format of parse_string matches train.trees'''
print()

In [13]:
parse_string

'(TOP (SQ (VBZ Does) (SQ* (NP (DT this) (NN flight)) (VP (VB serve) (NP_NN dinner)))) (PUNC ?))'

In [14]:
resulting_tree = trees.Tree.from_str(parse_string)
print(resulting_tree.pretty_print())

                TOP
        ┌────────┴─────────┐
        SQ                PUNC
  ┌─────┴─────┐            │
 VBZ         SQ*           ?
  │     ┌─────┴─────┐
Does    NP          VP
     ┌──┴──┐     ┌──┴───┐
     DT    NN    VB   NP_NN
     │     │     │      │
   this flight serve dinner
